In [1]:
# https://github.com/uchidalab/time_series_augmentation/blob/master/utils/augmentation.py

from os.path import dirname
import numpy as np
from sklearn import preprocessing
from Classifiers.OS_CNN.OS_CNN_easy_use import OS_CNN_easy_use
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

import torch
seed = 0
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
# np.random.seed(0)
# import random
# random.seed(0)


def TSC_data_loader(dataset_path,dataset_name):
    Train_dataset = np.loadtxt(
        dataset_path + '/' + dataset_name + '/' + dataset_name + '_TRAIN.tsv')
    Test_dataset = np.loadtxt(
        dataset_path + '/' + dataset_name + '/' + dataset_name + '_TEST.tsv')
    Train_dataset = Train_dataset.astype(np.float32)
    Test_dataset = Test_dataset.astype(np.float32)

    X_train = Train_dataset[:, 1:]
    y_train = Train_dataset[:, 0:1]

    X_test = Test_dataset[:, 1:]
    y_test = Test_dataset[:, 0:1]
    le = preprocessing.LabelEncoder()
    le.fit(np.squeeze(y_train, axis=1))
    y_train = le.transform(np.squeeze(y_train, axis=1))
    y_test = le.transform(np.squeeze(y_test, axis=1))
    return set_nan_to_zero(X_train), y_train, set_nan_to_zero(X_test), y_test


def set_nan_to_zero(a):
    where_are_NaNs = np.isnan(a)
    a[where_are_NaNs] = 0
    return a
# Datasets with performance problems
all_dataset_name_list = [
    "ProximalPhalanxOutlineCorrect",
    "LargeKitchenAppliances",
    "CricketY",
    "FaceAll",
    "CricketZ",
    "BeetleFly",
    "CricketX",
    "ChlorineConcentration",
    "Wine",
    "ProximalPhalanxOutlineAgeGroup",
    "ArrowHead",
    "FordB",
    "Adiac",
    "OliveOil",
    "ShapeletSim",
    "CinCECGTorso",
    "PhalangesOutlinesCorrect",
    "Beef",
    "MiddlePhalanxOutlineCorrect",
    "UWaveGestureLibraryX",
    "FiftyWords",
    "Lightning2",
    "Lightning7",
    "Worms",
    "ProximalPhalanxTW",
    "MedicalImages",
    "DistalPhalanxOutlineCorrect",
    "UWaveGestureLibraryZ",
    "DistalPhalanxOutlineAgeGroup",
    "UWaveGestureLibraryY",
    "WordSynonyms",
    "Computers",
    "ElectricDevices",
    "SmallKitchenAppliances",
    "Earthquakes",
    "Ham",
    "WormsTwoClass",
    "DistalPhalanxTW",
    "Herring"
]

short_dataset_name_list = ["Wine", "BeetleFly", "ArrowHead", "OliveOil", "Beef", "Lightning7", "Lightning2",
                     "DistalPhalanxOutlineAgeGroup", "DistalPhalanxTW", "Herring",
                     "ProximalPhalanxTW", "MiddlePhalanxOutlineCorrect", "ProximalPhalanxOutlineCorrect",
                     "ProximalPhalanxOutlineAgeGroup", "DistalPhalanxOutlineCorrect", "Ham"]
long_dataset_name_list = [dataset_name for dataset_name in all_dataset_name_list if dataset_name not in short_dataset_name_list]

In [2]:
print(len(long_dataset_name_list))
print(len(short_dataset_name_list))
print(len(all_dataset_name_list))

23
16
39


In [3]:
def jittering(data, proportion=0.05):
    std_dev = np.std(data)
    noise_level = proportion * std_dev
    noise = np.random.normal(0, noise_level, data.shape)
    return data + noise

def scaling(data, scale_factor=1.5):
    return data * scale_factor

def window_slicing(data, window_size=100):
    start = np.random.randint(0, data.shape[0] - window_size)
    return data[start:start + window_size]

from scipy.interpolate import interp1d

def time_warping(data, warping_factor=0.5):
    time_steps = np.arange(data.shape[0])
    new_time_steps = np.linspace(0, data.shape[0] - 1, int(data.shape[0] * warping_factor))

    warper = interp1d(time_steps, data, kind='linear', fill_value='extrapolate')
    return warper(new_time_steps)


def rotation(data, std_deviation_fraction=0.1):
    # Calculate the standard deviation of the data
    data_std_dev = np.std(data, axis=0)
    
    # Generate a rotation matrix with std deviation proportional to data_std_dev
    rotation_matrix = np.random.normal(0, std_deviation_fraction * data_std_dev, (data.shape[1], data.shape[1]))
    
    # Apply the rotation to the data
    rotated_data = np.dot(data, rotation_matrix)
    
    return rotated_data


def permutation(data, num_segments=5):
    permuted_data = np.zeros(data.shape)
    segment_size = data.shape[0] // num_segments

    indices = np.random.permutation(num_segments)
    for idx, segment_idx in enumerate(indices):
        start = idx * segment_size
        end = start + segment_size
        segment_start = segment_idx * segment_size
        segment_end = segment_start + segment_size

        permuted_data[start:end] = data[segment_start:segment_end]

    return permuted_data

def magnitude_warping(data, warp_factor=0.2):
    time_steps = np.arange(data.shape[0])
    warp_curve = np.sin(time_steps * warp_factor)
    return data * warp_curve

In [4]:
import torch
torch.cuda.is_available()


True

In [5]:
dataset_path = dirname("./Example_Datasets/UCRArchive_2018/")
dataset_name = "Wine"
X_train, y_train, X_test, y_test = TSC_data_loader(dataset_path, dataset_name)

In [6]:
print(dataset_name)
print('train data shape', X_train.shape)
print('train label shape',y_train.shape)
print('test data shape',X_test.shape)
print('test label shape',y_test.shape)
print('unique train label',np.unique(y_train))
print('unique test label',np.unique(y_test))

Wine
train data shape (57, 234)
train label shape (57,)
test data shape (54, 234)
test label shape (54,)
unique train label [0 1]
unique test label [0 1]


In [ ]:
results_df = [""]

# Original

In [13]:
# creat model and log save place,
Result_log_folder = './Example_Results_of_OS_CNN/OS_CNN_result_iter_0/'

dataset_name = "Wine"

model = OS_CNN_easy_use(
    Result_log_folder = Result_log_folder, # the Result_log_folder,
    dataset_name = dataset_name,           # dataset_name for creat log under Result_log_folder,
    device = "cuda:0",                # Gpu 
    max_epoch = 2000,                        # In our expirement the number is 2000 for keep it same with FCN for the example dataset 500 will be enough,
    paramenter_number_of_layer_list = [8*128, 5*128*256 + 2*256*128],
    )

model.fit(X_train, y_train, X_test, y_test)

y_predict = model.predict(X_test)

print('correct:',y_test)
print('predict:',y_predict)
acc = accuracy_score(y_predict, y_test)
print(acc)

code is running on  cuda:0
epoch = 49 lr =  0.001
train_acc=	 0.5789473684210527 	 test_acc=	 0.5 	 loss=	 0.6975501775741577
log saved at:
./Example_Results_of_OS_CNN/OS_CNN_result_iter_0/Wine/Wine_.txt
epoch = 99 lr =  0.001
train_acc=	 0.47368421052631576 	 test_acc=	 0.5 	 loss=	 0.6242873668670654
log saved at:
./Example_Results_of_OS_CNN/OS_CNN_result_iter_0/Wine/Wine_.txt
epoch = 149 lr =  0.001
train_acc=	 0.6491228070175439 	 test_acc=	 0.6111111111111112 	 loss=	 0.6935000419616699
log saved at:
./Example_Results_of_OS_CNN/OS_CNN_result_iter_0/Wine/Wine_.txt
epoch = 199 lr =  0.001
train_acc=	 0.631578947368421 	 test_acc=	 0.5555555555555556 	 loss=	 0.7695827484130859
log saved at:
./Example_Results_of_OS_CNN/OS_CNN_result_iter_0/Wine/Wine_.txt
epoch = 249 lr =  0.001
train_acc=	 0.5263157894736842 	 test_acc=	 0.5555555555555556 	 loss=	 0.7452268600463867
log saved at:
./Example_Results_of_OS_CNN/OS_CNN_result_iter_0/Wine/Wine_.txt
epoch = 299 lr =  0.0005
train_acc=	 0.7

# Scaling

In [18]:
X_train_scaled_loop = []
for x in X_train:
    X_train_scaled_loop.append(scaling(x))
X_train_scaled_loop = np.array(X_train_scaled_loop)
X_train_scaled_loop = X_train_scaled_loop.astype(np.float32)
X_train_scaled = scaling(X_train).astype(np.float32)
assert np.array_equal(X_train_scaled_loop, X_train_scaled)

In [88]:
# creat model and log save place,
Result_log_folder = './Example_Results_of_OS_CNN/OS_CNN_result_iter_0/'

dataset_name = "Wine_scaled"

model = OS_CNN_easy_use(
    Result_log_folder = Result_log_folder, # the Result_log_folder,
    dataset_name = dataset_name,           # dataset_name for creat log under Result_log_folder,
    device = "cuda:0",                # Gpu 
    max_epoch = 2000,                        # In our expirement the number is 2000 for keep it same with FCN for the example dataset 500 will be enough,
    paramenter_number_of_layer_list = [8*128, 5*128*256 + 2*256*128],
    )

model.fit(X_train_scaled, y_train, X_test, y_test)

y_predict = model.predict(X_test)

print('correct:',y_test)
print('predict:',y_predict)
acc = accuracy_score(y_predict, y_test)
print(acc)

code is running on  cuda:0
epoch = 49 lr =  0.001
train_acc=	 0.5789473684210527 	 test_acc=	 0.5 	 loss=	 0.7057485580444336
log saved at:
./Example_Results_of_OS_CNN/OS_CNN_result_iter_0/Wine_scaled/Wine_scaled_.txt
epoch = 99 lr =  0.001
train_acc=	 0.47368421052631576 	 test_acc=	 0.5 	 loss=	 0.686498761177063
log saved at:
./Example_Results_of_OS_CNN/OS_CNN_result_iter_0/Wine_scaled/Wine_scaled_.txt
epoch = 149 lr =  0.001
train_acc=	 0.6666666666666666 	 test_acc=	 0.5555555555555556 	 loss=	 0.650382399559021
log saved at:
./Example_Results_of_OS_CNN/OS_CNN_result_iter_0/Wine_scaled/Wine_scaled_.txt
epoch = 199 lr =  0.001
train_acc=	 0.5263157894736842 	 test_acc=	 0.5 	 loss=	 0.6788427829742432
log saved at:
./Example_Results_of_OS_CNN/OS_CNN_result_iter_0/Wine_scaled/Wine_scaled_.txt
epoch = 249 lr =  0.0005
train_acc=	 0.7192982456140351 	 test_acc=	 0.5 	 loss=	 0.5812201499938965
log saved at:
./Example_Results_of_OS_CNN/OS_CNN_result_iter_0/Wine_scaled/Wine_scaled_.txt


In [7]:
aug_percentage = 0.2 # 20 percent
aug_length = int(aug_percentage*len(X_train_scaled))

X_train_aug = np.vstack((X_train, X_train_scaled[:aug_length]))
y_train_aug = np.concatenate((y_train, y_train[:aug_length]))
y_train_aug.shape

(68,)

In [9]:
# creat model and log save place,
Result_log_folder = './Example_Results_of_OS_CNN/OS_CNN_result_iter_0/'

dataset_name = "Wine_scaled_aug"

model = OS_CNN_easy_use(
    Result_log_folder = Result_log_folder, # the Result_log_folder,
    dataset_name = dataset_name,           # dataset_name for creat log under Result_log_folder,
    device = "cuda:0",                # Gpu 
    max_epoch = 2000,                        # In our expirement the number is 2000 for keep it same with FCN for the example dataset 500 will be enough,
    paramenter_number_of_layer_list = [8*128, 5*128*256 + 2*256*128],
    )

model.fit(X_train_aug, y_train_aug, X_test, y_test)

y_predict = model.predict(X_test)

print('correct:',y_test)
print('predict:',y_predict)
acc = accuracy_score(y_predict, y_test)
print(acc)

code is running on  cuda:0
epoch = 49 lr =  0.001
train_acc=	 0.6029411764705882 	 test_acc=	 0.5185185185185185 	 loss=	 0.5115618705749512
log saved at:
./Example_Results_of_OS_CNN/OS_CNN_result_iter_0/Wine_scaled_aug/Wine_scaled_aug_.txt
epoch = 99 lr =  0.0005
train_acc=	 0.6029411764705882 	 test_acc=	 0.5370370370370371 	 loss=	 0.8562476634979248
log saved at:
./Example_Results_of_OS_CNN/OS_CNN_result_iter_0/Wine_scaled_aug/Wine_scaled_aug_.txt
epoch = 149 lr =  0.0005
train_acc=	 0.6470588235294118 	 test_acc=	 0.4444444444444444 	 loss=	 0.7476552128791809
log saved at:
./Example_Results_of_OS_CNN/OS_CNN_result_iter_0/Wine_scaled_aug/Wine_scaled_aug_.txt
epoch = 199 lr =  0.00025
train_acc=	 0.6911764705882353 	 test_acc=	 0.5925925925925926 	 loss=	 0.7300503849983215
log saved at:
./Example_Results_of_OS_CNN/OS_CNN_result_iter_0/Wine_scaled_aug/Wine_scaled_aug_.txt
epoch = 249 lr =  0.000125
train_acc=	 0.7352941176470589 	 test_acc=	 0.6296296296296297 	 loss=	 0.8882902860

# Jitter

In [63]:
X_train_jitter = []
for x in X_train:
    X_train_jitter.append(jittering(x))
X_train_jitter = np.array(X_train_jitter)
X_train_jitter = X_train_jitter.astype(np.float32)

In [67]:
# creat model and log save place,
Result_log_folder = './Example_Results_of_OS_CNN/OS_CNN_result_iter_0/'

dataset_name = "Wine_jitter"

model = OS_CNN_easy_use(
    Result_log_folder = Result_log_folder, # the Result_log_folder,
    dataset_name = dataset_name,           # dataset_name for creat log under Result_log_folder,
    device = "cuda:0",                # Gpu 
    max_epoch = 2000,                        # In our expirement the number is 2000 for keep it same with FCN for the example dataset 500 will be enough,
    paramenter_number_of_layer_list = [8*128, 5*128*256 + 2*256*128],
    )

model.fit(X_train_jitter, y_train, X_test, y_test)

y_predict = model.predict(X_test)

print('correct:',y_test)
print('predict:',y_predict)
acc = accuracy_score(y_predict, y_test)
print(acc)

code is running on  cuda:0
epoch = 49 lr =  0.001
train_acc=	 0.47368421052631576 	 test_acc=	 0.5 	 loss=	 0.6693470478057861
log saved at:
./Example_Results_of_OS_CNN/OS_CNN_result_iter_0/Wine_jitter/Wine_jitter_.txt
epoch = 99 lr =  0.001
train_acc=	 0.47368421052631576 	 test_acc=	 0.5 	 loss=	 0.3406524658203125
log saved at:
./Example_Results_of_OS_CNN/OS_CNN_result_iter_0/Wine_jitter/Wine_jitter_.txt
epoch = 149 lr =  0.001
train_acc=	 0.5263157894736842 	 test_acc=	 0.5555555555555556 	 loss=	 0.8765867948532104
log saved at:
./Example_Results_of_OS_CNN/OS_CNN_result_iter_0/Wine_jitter/Wine_jitter_.txt
epoch = 199 lr =  0.001
train_acc=	 1.0 	 test_acc=	 0.5 	 loss=	 0.06869316846132278
log saved at:
./Example_Results_of_OS_CNN/OS_CNN_result_iter_0/Wine_jitter/Wine_jitter_.txt
epoch = 249 lr =  0.001
train_acc=	 1.0 	 test_acc=	 0.5 	 loss=	 0.42430320382118225
log saved at:
./Example_Results_of_OS_CNN/OS_CNN_result_iter_0/Wine_jitter/Wine_jitter_.txt
epoch = 299 lr =  0.001
tr

In [68]:
# creat model and log save place,
Result_log_folder = './Example_Results_of_OS_CNN/OS_CNN_result_iter_0/'

dataset_name = "Wine"

model = OS_CNN_easy_use(
    Result_log_folder = Result_log_folder, # the Result_log_folder,
    dataset_name = dataset_name,           # dataset_name for creat log under Result_log_folder,
    device = "cuda:0",                # Gpu 
    max_epoch = 2000,                        # In our expirement the number is 2000 for keep it same with FCN for the example dataset 500 will be enough,
    paramenter_number_of_layer_list = [8*128, 5*128*256 + 2*256*128],
    )

model.fit(X_train, y_train, X_test, y_test)

y_predict = model.predict(X_test)

print('correct:',y_test)
print('predict:',y_predict)
acc = accuracy_score(y_predict, y_test)
print(acc)

code is running on  cuda:0
epoch = 49 lr =  0.001
train_acc=	 0.47368421052631576 	 test_acc=	 0.5 	 loss=	 0.6963512897491455
log saved at:
./Example_Results_of_OS_CNN/OS_CNN_result_iter_0/Wine_jitter/Wine_jitter_.txt
epoch = 99 lr =  0.001
train_acc=	 0.5263157894736842 	 test_acc=	 0.5 	 loss=	 0.8184735774993896
log saved at:
./Example_Results_of_OS_CNN/OS_CNN_result_iter_0/Wine_jitter/Wine_jitter_.txt
epoch = 149 lr =  0.001
train_acc=	 0.6666666666666666 	 test_acc=	 0.7777777777777778 	 loss=	 0.6748399138450623
log saved at:
./Example_Results_of_OS_CNN/OS_CNN_result_iter_0/Wine_jitter/Wine_jitter_.txt
epoch = 199 lr =  0.001
train_acc=	 0.5964912280701754 	 test_acc=	 0.6481481481481481 	 loss=	 0.5061530470848083
log saved at:
./Example_Results_of_OS_CNN/OS_CNN_result_iter_0/Wine_jitter/Wine_jitter_.txt
epoch = 249 lr =  0.0005
train_acc=	 0.7719298245614035 	 test_acc=	 0.8333333333333334 	 loss=	 0.4693072438240051
log saved at:
./Example_Results_of_OS_CNN/OS_CNN_result_iter

In [83]:
aug_percentage = 0.2 # 20 percent
aug_length = int(aug_percentage*len(X_train_jitter))

X_train_aug = np.vstack((X_train, X_train_jitter[:aug_length]))
y_train_aug = np.concatenate((y_train, y_train[:aug_length]))
y_train_aug.shape

(68,)

In [84]:
# creat model and log save place,
Result_log_folder = './Example_Results_of_OS_CNN/OS_CNN_result_iter_0/'

dataset_name = "Wine_jitter_aug"

model = OS_CNN_easy_use(
    Result_log_folder = Result_log_folder, # the Result_log_folder,
    dataset_name = dataset_name,           # dataset_name for creat log under Result_log_folder,
    device = "cuda:0",                # Gpu 
    max_epoch = 2000,                        # In our expirement the number is 2000 for keep it same with FCN for the example dataset 500 will be enough,
    paramenter_number_of_layer_list = [8*128, 5*128*256 + 2*256*128],
    )

model.fit(X_train_aug, y_train_aug, X_test, y_test)

y_predict = model.predict(X_test)

print('correct:',y_test)
print('predict:',y_predict)
acc = accuracy_score(y_predict, y_test)
print(acc)

code is running on  cuda:0
epoch = 49 lr =  0.001
train_acc=	 0.5588235294117647 	 test_acc=	 0.5 	 loss=	 0.9909926056861877
log saved at:
./Example_Results_of_OS_CNN/OS_CNN_result_iter_0/Wine_jitter_aug/Wine_jitter_aug_.txt
epoch = 99 lr =  0.001
train_acc=	 0.6029411764705882 	 test_acc=	 0.5555555555555556 	 loss=	 0.718720555305481
log saved at:
./Example_Results_of_OS_CNN/OS_CNN_result_iter_0/Wine_jitter_aug/Wine_jitter_aug_.txt
epoch = 149 lr =  0.0005
train_acc=	 0.6911764705882353 	 test_acc=	 0.6111111111111112 	 loss=	 0.9773963689804077
log saved at:
./Example_Results_of_OS_CNN/OS_CNN_result_iter_0/Wine_jitter_aug/Wine_jitter_aug_.txt
epoch = 199 lr =  0.0005
train_acc=	 0.7058823529411765 	 test_acc=	 0.6111111111111112 	 loss=	 0.6823687553405762
log saved at:
./Example_Results_of_OS_CNN/OS_CNN_result_iter_0/Wine_jitter_aug/Wine_jitter_aug_.txt
epoch = 249 lr =  0.00025
train_acc=	 0.75 	 test_acc=	 0.7222222222222222 	 loss=	 0.8554990291595459
log saved at:
./Example_Re

# Window Slicing

In [41]:
X_train_window_loop = []
window_size = int(X_train.shape[1]*0.95)
for x in X_train:
    X_train_window_loop.append(window_slicing(x, window_size=window_size))
X_train_window_loop = np.array(X_train_window_loop)
X_train_window_loop = X_train_window_loop.astype(np.float32)
# X_train_window = window_slicing(X_train).astype(np.float32)
# assert np.array_equal(X_train_window_loop, X_train_window)

In [42]:
window_size

222

In [43]:
X_train_window_loop.shape

(57, 222)

In [45]:
# creat model and log save place,
Result_log_folder = './Example_Results_of_OS_CNN/OS_CNN_result_iter_0/'

dataset_name = "Wine_window"

model = OS_CNN_easy_use(
    Result_log_folder = Result_log_folder, # the Result_log_folder,
    dataset_name = dataset_name,           # dataset_name for creat log under Result_log_folder,
    device = "cuda:0",                # Gpu 
    max_epoch = 2000,                        # In our expirement the number is 2000 for keep it same with FCN for the example dataset 500 will be enough,
    paramenter_number_of_layer_list = [8*128, 5*128*256 + 2*256*128],
    )

model.fit(X_train_window_loop, y_train, X_test, y_test)

y_predict = model.predict(X_test)

print('correct:',y_test)
print('predict:',y_predict)
acc = accuracy_score(y_predict, y_test)
print(acc)

code is running on  cuda:0
epoch = 49 lr =  0.001
train_acc=	 0.5964912280701754 	 test_acc=	 0.5 	 loss=	 0.6976895332336426
log saved at:
./Example_Results_of_OS_CNN/OS_CNN_result_iter_0/Wine_window/Wine_window_.txt
epoch = 99 lr =  0.001
train_acc=	 0.47368421052631576 	 test_acc=	 0.5 	 loss=	 0.7101874351501465
log saved at:
./Example_Results_of_OS_CNN/OS_CNN_result_iter_0/Wine_window/Wine_window_.txt
epoch = 149 lr =  0.001
train_acc=	 0.6491228070175439 	 test_acc=	 0.5555555555555556 	 loss=	 0.5958192348480225
log saved at:
./Example_Results_of_OS_CNN/OS_CNN_result_iter_0/Wine_window/Wine_window_.txt
epoch = 199 lr =  0.001
train_acc=	 0.6491228070175439 	 test_acc=	 0.5740740740740741 	 loss=	 0.730559229850769
log saved at:
./Example_Results_of_OS_CNN/OS_CNN_result_iter_0/Wine_window/Wine_window_.txt
epoch = 249 lr =  0.0005
train_acc=	 0.6491228070175439 	 test_acc=	 0.5185185185185185 	 loss=	 0.7735893726348877
log saved at:
./Example_Results_of_OS_CNN/OS_CNN_result_iter_

In [46]:
aug_percentage = 0.2 # 20 percent
aug_length = int(aug_percentage*len(X_train_window_loop))

X_train_aug = np.vstack((X_train, X_train_window_loop[:aug_length]))
y_train_aug = np.concatenate((y_train, y_train[:aug_length]))
y_train_aug.shape

ValueError: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 234 and the array at index 1 has size 222

In [ ]:

# creat model and log save place,
Result_log_folder = './Example_Results_of_OS_CNN/OS_CNN_result_iter_0/'

dataset_name = "Wine_window_aug"

model = OS_CNN_easy_use(
    Result_log_folder = Result_log_folder, # the Result_log_folder,
    dataset_name = dataset_name,           # dataset_name for creat log under Result_log_folder,
    device = "cuda:0",                # Gpu 
    max_epoch = 2000,                        # In our expirement the number is 2000 for keep it same with FCN for the example dataset 500 will be enough,
    paramenter_number_of_layer_list = [8*128, 5*128*256 + 2*256*128],
    )

model.fit(X_train_aug, y_train_aug, X_test, y_test)

y_predict = model.predict(X_test)

print('correct:',y_test)
print('predict:',y_predict)
acc = accuracy_score(y_predict, y_test)
print(acc)

# Time Warping

In [53]:
X_train_warped_loop = []
for x in X_train:
    X_train_warped_loop.append(time_warping(x, warping_factor=0.9))
X_train_warped_loop = np.array(X_train_warped_loop)
X_train_warped_loop = X_train_warped_loop.astype(np.float32)
X_train_warped_loop.shape
# X_train_warped = scaling(X_train).astype(np.float32)
# assert np.array_equal(X_train_warped_loop, X_train_warped)

(57, 210)

In [54]:
# creat model and log save place,
Result_log_folder = './Example_Results_of_OS_CNN/OS_CNN_result_iter_0/'

dataset_name = "Wine_warped"

model = OS_CNN_easy_use(
    Result_log_folder = Result_log_folder, # the Result_log_folder,
    dataset_name = dataset_name,           # dataset_name for creat log under Result_log_folder,
    device = "cuda:0",                # Gpu 
    max_epoch = 2000,                        # In our expirement the number is 2000 for keep it same with FCN for the example dataset 500 will be enough,
    paramenter_number_of_layer_list = [8*128, 5*128*256 + 2*256*128],
    )

model.fit(X_train_warped_loop, y_train, X_test, y_test)

y_predict = model.predict(X_test)

print('correct:',y_test)
print('predict:',y_predict)
acc = accuracy_score(y_predict, y_test)
print(acc)

code is running on  cuda:0
epoch = 49 lr =  0.001
train_acc=	 0.5263157894736842 	 test_acc=	 0.5 	 loss=	 0.684441089630127
log saved at:
./Example_Results_of_OS_CNN/OS_CNN_result_iter_0/Wine_warped/Wine_warped_.txt
epoch = 99 lr =  0.0005
train_acc=	 0.5263157894736842 	 test_acc=	 0.5 	 loss=	 0.8242714405059814
log saved at:
./Example_Results_of_OS_CNN/OS_CNN_result_iter_0/Wine_warped/Wine_warped_.txt
epoch = 149 lr =  0.0005
train_acc=	 0.6491228070175439 	 test_acc=	 0.5 	 loss=	 0.819429337978363
log saved at:
./Example_Results_of_OS_CNN/OS_CNN_result_iter_0/Wine_warped/Wine_warped_.txt
epoch = 199 lr =  0.00025
train_acc=	 0.7192982456140351 	 test_acc=	 0.5 	 loss=	 0.6791395545005798
log saved at:
./Example_Results_of_OS_CNN/OS_CNN_result_iter_0/Wine_warped/Wine_warped_.txt
epoch = 249 lr =  0.000125
train_acc=	 0.7368421052631579 	 test_acc=	 0.5 	 loss=	 0.7569926381111145
log saved at:
./Example_Results_of_OS_CNN/OS_CNN_result_iter_0/Wine_warped/Wine_warped_.txt
epoch = 29

# Window warping

In [107]:
def window_warp(x, window_ratio=0.1, scales=[0.5, 2.]):
    # https://halshs.archives-ouvertes.fr/halshs-01357973/document
    warp_scales = np.random.choice(scales, x.shape[0])
    warp_size = np.ceil(window_ratio*x.shape[1]).astype(int)
    window_steps = np.arange(warp_size)
        
    window_starts = np.random.randint(low=1, high=x.shape[1]-warp_size-1, size=(x.shape[0])).astype(int)
    window_ends = (window_starts + warp_size).astype(int)
            
    ret = np.zeros_like(x)
    for i, pat in enumerate(x):
        for dim in range(x.shape[2]):
            start_seg = pat[:window_starts[i],dim]
            window_seg = np.interp(np.linspace(0, warp_size-1, num=int(warp_size*warp_scales[i])), window_steps, pat[window_starts[i]:window_ends[i],dim])
            end_seg = pat[window_ends[i]:,dim]
            warped = np.concatenate((start_seg, window_seg, end_seg))                
            ret[i,:,dim] = np.interp(np.arange(x.shape[1]), np.linspace(0, x.shape[1]-1., num=warped.size), warped).T
    return ret



In [108]:
X_train_window_warping = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_train_window_warping = window_warp(X_train_window_warping)
X_train_window_warping = X_train_window_warping.reshape((X_train_window_warping.shape[0], X_train_window_warping.shape[1]))
X_train_window_warping.shape

(57, 234)

In [109]:
# creat model and log save place,
Result_log_folder = './Example_Results_of_OS_CNN/OS_CNN_result_iter_0/'

dataset_name = "Wine_window_warping"

model = OS_CNN_easy_use(
    Result_log_folder = Result_log_folder, # the Result_log_folder,
    dataset_name = dataset_name,           # dataset_name for creat log under Result_log_folder,
    device = "cuda:0",                # Gpu 
    max_epoch = 2000,                        # In our expirement the number is 2000 for keep it same with FCN for the example dataset 500 will be enough,
    paramenter_number_of_layer_list = [8*128, 5*128*256 + 2*256*128],
    )

model.fit(X_train_window_warping, y_train, X_test, y_test)

y_predict = model.predict(X_test)

print('correct:',y_test)
print('predict:',y_predict)
acc = accuracy_score(y_predict, y_test)
print(acc)

code is running on  cuda:0
epoch = 49 lr =  0.001
train_acc=	 0.6666666666666666 	 test_acc=	 0.5 	 loss=	 0.5681410431861877
log saved at:
./Example_Results_of_OS_CNN/OS_CNN_result_iter_0/Wine_window_warping/Wine_window_warping_.txt
epoch = 99 lr =  0.001
train_acc=	 0.7017543859649122 	 test_acc=	 0.5 	 loss=	 0.5506792664527893
log saved at:
./Example_Results_of_OS_CNN/OS_CNN_result_iter_0/Wine_window_warping/Wine_window_warping_.txt
epoch = 149 lr =  0.001
train_acc=	 0.8070175438596491 	 test_acc=	 0.5 	 loss=	 0.17749711871147156
log saved at:
./Example_Results_of_OS_CNN/OS_CNN_result_iter_0/Wine_window_warping/Wine_window_warping_.txt
epoch = 199 lr =  0.001
train_acc=	 0.6666666666666666 	 test_acc=	 0.5 	 loss=	 0.1439720094203949
log saved at:
./Example_Results_of_OS_CNN/OS_CNN_result_iter_0/Wine_window_warping/Wine_window_warping_.txt
epoch = 249 lr =  0.0005
train_acc=	 0.6491228070175439 	 test_acc=	 0.5 	 loss=	 0.6062359809875488
log saved at:
./Example_Results_of_OS_CNN

In [110]:
aug_percentage = 0.2 # 20 percent
aug_length = int(aug_percentage*len(X_train_window_warping))

X_train_aug = np.vstack((X_train, X_train_window_warping[:aug_length]))
y_train_aug = np.concatenate((y_train, y_train[:aug_length]))
y_train_aug.shape

(68,)

In [111]:
# creat model and log save place,
Result_log_folder = './Example_Results_of_OS_CNN/OS_CNN_result_iter_0/'

dataset_name = "Wine_window_warping_aug"

model = OS_CNN_easy_use(
    Result_log_folder = Result_log_folder, # the Result_log_folder,
    dataset_name = dataset_name,           # dataset_name for creat log under Result_log_folder,
    device = "cuda:0",                # Gpu 
    max_epoch = 2000,                        # In our expirement the number is 2000 for keep it same with FCN for the example dataset 500 will be enough,
    paramenter_number_of_layer_list = [8*128, 5*128*256 + 2*256*128],
    )

model.fit(X_train_aug, y_train_aug, X_test, y_test)

y_predict = model.predict(X_test)

print('correct:',y_test)
print('predict:',y_predict)
acc = accuracy_score(y_predict, y_test)
print(acc)

code is running on  cuda:0
epoch = 49 lr =  0.001
train_acc=	 0.5441176470588235 	 test_acc=	 0.5 	 loss=	 0.6399391889572144
log saved at:
./Example_Results_of_OS_CNN/OS_CNN_result_iter_0/Wine_window_warping_aug/Wine_window_warping_aug_.txt
epoch = 99 lr =  0.001
train_acc=	 0.6029411764705882 	 test_acc=	 0.5 	 loss=	 1.128157377243042
log saved at:
./Example_Results_of_OS_CNN/OS_CNN_result_iter_0/Wine_window_warping_aug/Wine_window_warping_aug_.txt
epoch = 149 lr =  0.0005
train_acc=	 0.5441176470588235 	 test_acc=	 0.5 	 loss=	 0.46388429403305054
log saved at:
./Example_Results_of_OS_CNN/OS_CNN_result_iter_0/Wine_window_warping_aug/Wine_window_warping_aug_.txt
epoch = 199 lr =  0.00025
train_acc=	 0.6176470588235294 	 test_acc=	 0.5555555555555556 	 loss=	 1.1225035190582275
log saved at:
./Example_Results_of_OS_CNN/OS_CNN_result_iter_0/Wine_window_warping_aug/Wine_window_warping_aug_.txt
epoch = 249 lr =  0.000125
train_acc=	 0.6911764705882353 	 test_acc=	 0.6111111111111112 	 l

# Rotation

In [15]:
def rotation(x):
    flip = np.random.choice([-1, 1], size=(x.shape[0],x.shape[2]))
    rotate_axis = np.arange(x.shape[2])
    np.random.shuffle(rotate_axis)    
    return flip[:,np.newaxis,:] * x[:,:,rotate_axis]

In [17]:
X_train_rotation = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_train_rotation = rotation(X_train_rotation)
X_train_rotation = X_train_rotation.reshape((X_train_rotation.shape[0], X_train_rotation.shape[1]))
X_train_rotation = X_train_rotation.astype(np.float32)

In [18]:
# creat model and log save place,
Result_log_folder = './Example_Results_of_OS_CNN/OS_CNN_result_iter_0/'

dataset_name = "Wine_rotation"

model = OS_CNN_easy_use(
    Result_log_folder = Result_log_folder, # the Result_log_folder,
    dataset_name = dataset_name,           # dataset_name for creat log under Result_log_folder,
    device = "cuda:0",                # Gpu 
    max_epoch = 2000,                        # In our expirement the number is 2000 for keep it same with FCN for the example dataset 500 will be enough,
    paramenter_number_of_layer_list = [8*128, 5*128*256 + 2*256*128],
    )

model.fit(X_train_rotation, y_train, X_test, y_test)

y_predict = model.predict(X_test)

print('correct:',y_test)
print('predict:',y_predict)
acc = accuracy_score(y_predict, y_test)
print(acc)

code is running on  cuda:0
epoch = 49 lr =  0.001
train_acc=	 0.5964912280701754 	 test_acc=	 0.5 	 loss=	 0.8607219457626343
log saved at:
./Example_Results_of_OS_CNN/OS_CNN_result_iter_0/Wine_rotation/Wine_rotation_.txt
epoch = 99 lr =  0.001
train_acc=	 0.5964912280701754 	 test_acc=	 0.5 	 loss=	 0.5035196542739868
log saved at:
./Example_Results_of_OS_CNN/OS_CNN_result_iter_0/Wine_rotation/Wine_rotation_.txt
epoch = 149 lr =  0.0005
train_acc=	 0.5964912280701754 	 test_acc=	 0.5 	 loss=	 0.9042139053344727
log saved at:
./Example_Results_of_OS_CNN/OS_CNN_result_iter_0/Wine_rotation/Wine_rotation_.txt
epoch = 199 lr =  0.00025
train_acc=	 0.5964912280701754 	 test_acc=	 0.5 	 loss=	 0.8016318082809448
log saved at:
./Example_Results_of_OS_CNN/OS_CNN_result_iter_0/Wine_rotation/Wine_rotation_.txt
epoch = 249 lr =  0.00025
train_acc=	 0.5964912280701754 	 test_acc=	 0.5 	 loss=	 0.47033366560935974
log saved at:
./Example_Results_of_OS_CNN/OS_CNN_result_iter_0/Wine_rotation/Wine_rot

In [19]:
aug_percentage = 0.2 # 20 percent
aug_length = int(aug_percentage*len(X_train_rotation))

X_train_aug = np.vstack((X_train, X_train_rotation[:aug_length]))
y_train_aug = np.concatenate((y_train, y_train[:aug_length]))
y_train_aug.shape

(68,)

In [20]:
# creat model and log save place,
Result_log_folder = './Example_Results_of_OS_CNN/OS_CNN_result_iter_0/'

dataset_name = "Wine_rotation_aug"

model = OS_CNN_easy_use(
    Result_log_folder = Result_log_folder, # the Result_log_folder,
    dataset_name = dataset_name,           # dataset_name for creat log under Result_log_folder,
    device = "cuda:0",                # Gpu 
    max_epoch = 2000,                        # In our expirement the number is 2000 for keep it same with FCN for the example dataset 500 will be enough,
    paramenter_number_of_layer_list = [8*128, 5*128*256 + 2*256*128],
    )

model.fit(X_train_aug, y_train_aug, X_test, y_test)

y_predict = model.predict(X_test)

print('correct:',y_test)
print('predict:',y_predict)
acc = accuracy_score(y_predict, y_test)
print(acc)

code is running on  cuda:0
epoch = 49 lr =  0.001
train_acc=	 0.6029411764705882 	 test_acc=	 0.5 	 loss=	 0.5069422721862793
log saved at:
./Example_Results_of_OS_CNN/OS_CNN_result_iter_0/Wine_rotation_aug/Wine_rotation_aug_.txt
epoch = 99 lr =  0.0005
train_acc=	 0.6911764705882353 	 test_acc=	 0.6111111111111112 	 loss=	 0.9640601873397827
log saved at:
./Example_Results_of_OS_CNN/OS_CNN_result_iter_0/Wine_rotation_aug/Wine_rotation_aug_.txt
epoch = 149 lr =  0.00025
train_acc=	 0.7058823529411765 	 test_acc=	 0.7037037037037037 	 loss=	 0.5878884792327881
log saved at:
./Example_Results_of_OS_CNN/OS_CNN_result_iter_0/Wine_rotation_aug/Wine_rotation_aug_.txt
epoch = 199 lr =  0.000125
train_acc=	 0.6911764705882353 	 test_acc=	 0.5555555555555556 	 loss=	 0.9176710844039917
log saved at:
./Example_Results_of_OS_CNN/OS_CNN_result_iter_0/Wine_rotation_aug/Wine_rotation_aug_.txt
epoch = 249 lr =  0.0001
train_acc=	 0.6911764705882353 	 test_acc=	 0.6111111111111112 	 loss=	 1.192322731

# Magnitude warping

In [94]:
def magnitude_warp(x, sigma=0.2, knot=4):
    from scipy.interpolate import CubicSpline
    orig_steps = np.arange(x.shape[1])
    
    random_warps = np.random.normal(loc=1.0, scale=sigma, size=(x.shape[0], knot+2, x.shape[2]))
    warp_steps = (np.ones((x.shape[2],1))*(np.linspace(0, x.shape[1]-1., num=knot+2))).T
    ret = np.zeros_like(x)
    for i, pat in enumerate(x):
        warper = np.array([CubicSpline(warp_steps[:,dim], random_warps[i,:,dim])(orig_steps) for dim in range(x.shape[2])]).T
        ret[i] = pat * warper

    return ret

In [100]:
X_train_mag_warping = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_train_mag_warping = magnitude_warp(X_train_mag_warping)
X_train_mag_warping = X_train_mag_warping.reshape((X_train_mag_warping.shape[0], X_train_mag_warping.shape[1]))
# X_train_mag_warping_loop = []
# for x in X_train:
#     X_train_mag_warping_loop.append(magnitude_warping(x, warp_factor=0.01))
# X_train_mag_warping_loop = np.array(X_train_mag_warping_loop)
# X_train_mag_warping_loop = X_train_mag_warping_loop.astype(np.float32)
# X_train_mag_warping_loop.shape

In [106]:
# creat model and log save place,
Result_log_folder = './Example_Results_of_OS_CNN/OS_CNN_result_iter_0/'

dataset_name = "Wine_mag_warping"

model = OS_CNN_easy_use(
    Result_log_folder = Result_log_folder, # the Result_log_folder,
    dataset_name = dataset_name,           # dataset_name for creat log under Result_log_folder,
    device = "cuda:0",                # Gpu 
    max_epoch = 2000,                        # In our expirement the number is 2000 for keep it same with FCN for the example dataset 500 will be enough,
    paramenter_number_of_layer_list = [8*128, 5*128*256 + 2*256*128],
    )

model.fit(X_train_mag_warping, y_train, X_test, y_test)

y_predict = model.predict(X_test)

print('correct:',y_test)
print('predict:',y_predict)
acc = accuracy_score(y_predict, y_test)
print(acc)

code is running on  cuda:0
epoch = 49 lr =  0.001
train_acc=	 0.7017543859649122 	 test_acc=	 0.5 	 loss=	 0.8428773283958435
log saved at:
./Example_Results_of_OS_CNN/OS_CNN_result_iter_0/Wine_mag_warping/Wine_mag_warping_.txt
epoch = 99 lr =  0.001
train_acc=	 0.8245614035087719 	 test_acc=	 0.5 	 loss=	 0.19768860936164856
log saved at:
./Example_Results_of_OS_CNN/OS_CNN_result_iter_0/Wine_mag_warping/Wine_mag_warping_.txt
epoch = 149 lr =  0.0005
train_acc=	 0.8070175438596491 	 test_acc=	 0.5 	 loss=	 0.7967120409011841
log saved at:
./Example_Results_of_OS_CNN/OS_CNN_result_iter_0/Wine_mag_warping/Wine_mag_warping_.txt
epoch = 199 lr =  0.0005
train_acc=	 0.8421052631578947 	 test_acc=	 0.5740740740740741 	 loss=	 0.4566896855831146
log saved at:
./Example_Results_of_OS_CNN/OS_CNN_result_iter_0/Wine_mag_warping/Wine_mag_warping_.txt
epoch = 249 lr =  0.0005
train_acc=	 0.8421052631578947 	 test_acc=	 0.5 	 loss=	 0.12598581612110138
log saved at:
./Example_Results_of_OS_CNN/OS_CN

# wdba

In [114]:
#https://github.com/uchidalab/time_series_augmentation/blob/master/utils/augmentation.py
from tqdm import tqdm
def wdba(x, labels, batch_size=6, slope_constraint="symmetric", use_window=True, verbose=0):
    # https://ieeexplore.ieee.org/document/8215569
    # use verbose = -1 to turn off warnings    
    # slope_constraint is for DTW. "symmetric" or "asymmetric"
    
    import utils.dtw as dtw
    
    if use_window:
        window = np.ceil(x.shape[1] / 10.).astype(int)
    else:
        window = None
    orig_steps = np.arange(x.shape[1])
    l = np.argmax(labels, axis=1) if labels.ndim > 1 else labels
        
    ret = np.zeros_like(x)
    for i in tqdm(range(ret.shape[0])):
        # get the same class as i
        choices = np.where(l == l[i])[0]
        if choices.size > 0:        
            # pick random intra-class pattern
            k = min(choices.size, batch_size)
            random_prototypes = x[np.random.choice(choices, k, replace=False)]
            
            # calculate dtw between all
            dtw_matrix = np.zeros((k, k))
            for p, prototype in enumerate(random_prototypes):
                for s, sample in enumerate(random_prototypes):
                    if p == s:
                        dtw_matrix[p, s] = 0.
                    else:
                        dtw_matrix[p, s] = dtw.dtw(prototype, sample, dtw.RETURN_VALUE, slope_constraint=slope_constraint, window=window)
                        
            # get medoid
            medoid_id = np.argsort(np.sum(dtw_matrix, axis=1))[0]
            nearest_order = np.argsort(dtw_matrix[medoid_id])
            medoid_pattern = random_prototypes[medoid_id]
            
            # start weighted DBA
            average_pattern = np.zeros_like(medoid_pattern)
            weighted_sums = np.zeros((medoid_pattern.shape[0]))
            for nid in nearest_order:
                if nid == medoid_id or dtw_matrix[medoid_id, nearest_order[1]] == 0.:
                    average_pattern += medoid_pattern 
                    weighted_sums += np.ones_like(weighted_sums) 
                else:
                    path = dtw.dtw(medoid_pattern, random_prototypes[nid], dtw.RETURN_PATH, slope_constraint=slope_constraint, window=window)
                    dtw_value = dtw_matrix[medoid_id, nid]
                    warped = random_prototypes[nid, path[1]]
                    weight = np.exp(np.log(0.5)*dtw_value/dtw_matrix[medoid_id, nearest_order[1]])
                    average_pattern[path[0]] += weight * warped
                    weighted_sums[path[0]] += weight 
            
            ret[i,:] = average_pattern / weighted_sums[:,np.newaxis]
        else:
            if verbose > -1:
                print("There is only one pattern of class %d, skipping pattern average"%l[i])
            ret[i,:] = x[i]
    return ret

In [115]:
X_train_wdba = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_train_wdba = wdba(X_train_wdba, y_train)
X_train_wdba = X_train_wdba.reshape((X_train_wdba.shape[0], X_train_wdba.shape[1]))
X_train_wdba.shape

100%|██████████| 57/57 [00:12<00:00,  4.65it/s]


(57, 234)

In [116]:
# creat model and log save place,
Result_log_folder = './Example_Results_of_OS_CNN/OS_CNN_result_iter_0/'

dataset_name = "Wine_wdba"

model = OS_CNN_easy_use(
    Result_log_folder = Result_log_folder, # the Result_log_folder,
    dataset_name = dataset_name,           # dataset_name for creat log under Result_log_folder,
    device = "cuda:0",                # Gpu 
    max_epoch = 2000,                        # In our expirement the number is 2000 for keep it same with FCN for the example dataset 500 will be enough,
    paramenter_number_of_layer_list = [8*128, 5*128*256 + 2*256*128],
    )

model.fit(X_train_wdba, y_train, X_test, y_test)

y_predict = model.predict(X_test)

print('correct:',y_test)
print('predict:',y_predict)
acc = accuracy_score(y_predict, y_test)
print(acc)

code is running on  cuda:0
epoch = 49 lr =  0.001
train_acc=	 0.47368421052631576 	 test_acc=	 0.5 	 loss=	 0.6961444020271301
log saved at:
./Example_Results_of_OS_CNN/OS_CNN_result_iter_0/Wine_wdba/Wine_wdba_.txt
epoch = 99 lr =  0.001
train_acc=	 0.5263157894736842 	 test_acc=	 0.5555555555555556 	 loss=	 0.6511180400848389
log saved at:
./Example_Results_of_OS_CNN/OS_CNN_result_iter_0/Wine_wdba/Wine_wdba_.txt
epoch = 149 lr =  0.0005
train_acc=	 0.5087719298245614 	 test_acc=	 0.48148148148148145 	 loss=	 0.20719406008720398
log saved at:
./Example_Results_of_OS_CNN/OS_CNN_result_iter_0/Wine_wdba/Wine_wdba_.txt
epoch = 199 lr =  0.0005
train_acc=	 0.8421052631578947 	 test_acc=	 0.5555555555555556 	 loss=	 0.48389288783073425
log saved at:
./Example_Results_of_OS_CNN/OS_CNN_result_iter_0/Wine_wdba/Wine_wdba_.txt
epoch = 249 lr =  0.0005
train_acc=	 0.6666666666666666 	 test_acc=	 0.4444444444444444 	 loss=	 0.6368327140808105
log saved at:
./Example_Results_of_OS_CNN/OS_CNN_result_

In [117]:
aug_percentage = 0.2 # 20 percent
aug_length = int(aug_percentage*len(X_train_wdba))

X_train_aug = np.vstack((X_train, X_train_wdba[:aug_length]))
y_train_aug = np.concatenate((y_train, y_train[:aug_length]))
y_train_aug.shape

(68,)

In [118]:
# creat model and log save place,
Result_log_folder = './Example_Results_of_OS_CNN/OS_CNN_result_iter_0/'

dataset_name = "Wine_wdba_aug"

model = OS_CNN_easy_use(
    Result_log_folder = Result_log_folder, # the Result_log_folder,
    dataset_name = dataset_name,           # dataset_name for creat log under Result_log_folder,
    device = "cuda:0",                # Gpu 
    max_epoch = 2000,                        # In our expirement the number is 2000 for keep it same with FCN for the example dataset 500 will be enough,
    paramenter_number_of_layer_list = [8*128, 5*128*256 + 2*256*128],
    )

model.fit(X_train_aug, y_train_aug, X_test, y_test)

y_predict = model.predict(X_test)

print('correct:',y_test)
print('predict:',y_predict)
acc = accuracy_score(y_predict, y_test)
print(acc)

code is running on  cuda:0
epoch = 49 lr =  0.001
train_acc=	 0.6470588235294118 	 test_acc=	 0.5 	 loss=	 0.728074848651886
log saved at:
./Example_Results_of_OS_CNN/OS_CNN_result_iter_0/Wine_wdba_aug/Wine_wdba_aug_.txt
epoch = 99 lr =  0.0005
train_acc=	 0.39705882352941174 	 test_acc=	 0.5 	 loss=	 1.0029951333999634
log saved at:
./Example_Results_of_OS_CNN/OS_CNN_result_iter_0/Wine_wdba_aug/Wine_wdba_aug_.txt
epoch = 149 lr =  0.0005
train_acc=	 0.7205882352941176 	 test_acc=	 0.6111111111111112 	 loss=	 0.6696425676345825
log saved at:
./Example_Results_of_OS_CNN/OS_CNN_result_iter_0/Wine_wdba_aug/Wine_wdba_aug_.txt
epoch = 199 lr =  0.0005
train_acc=	 0.4117647058823529 	 test_acc=	 0.5185185185185185 	 loss=	 0.5951530337333679
log saved at:
./Example_Results_of_OS_CNN/OS_CNN_result_iter_0/Wine_wdba_aug/Wine_wdba_aug_.txt
epoch = 249 lr =  0.00025
train_acc=	 0.75 	 test_acc=	 0.7962962962962963 	 loss=	 1.0375641584396362
log saved at:
./Example_Results_of_OS_CNN/OS_CNN_result

# SPAWNER

In [119]:
def jitter(x, sigma=0.03):
    # https://arxiv.org/pdf/1706.00527.pdf
    return x + np.random.normal(loc=0., scale=sigma, size=x.shape)

def spawner(x, labels, sigma=0.05, verbose=0):
    # https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6983028/
    # use verbose=-1 to turn off warnings
    # use verbose=1 to print out figures
    
    import utils.dtw as dtw
    random_points = np.random.randint(low=1, high=x.shape[1]-1, size=x.shape[0])
    window = np.ceil(x.shape[1] / 10.).astype(int)
    orig_steps = np.arange(x.shape[1])
    l = np.argmax(labels, axis=1) if labels.ndim > 1 else labels
    
    ret = np.zeros_like(x)
    for i, pat in enumerate(tqdm(x)):
        # guarentees that same one isnt selected
        choices = np.delete(np.arange(x.shape[0]), i)
        # remove ones of different classes
        choices = np.where(l[choices] == l[i])[0]
        if choices.size > 0:     
            random_sample = x[np.random.choice(choices)]
            # SPAWNER splits the path into two randomly
            path1 = dtw.dtw(pat[:random_points[i]], random_sample[:random_points[i]], dtw.RETURN_PATH, slope_constraint="symmetric", window=window)
            path2 = dtw.dtw(pat[random_points[i]:], random_sample[random_points[i]:], dtw.RETURN_PATH, slope_constraint="symmetric", window=window)
            combined = np.concatenate((np.vstack(path1), np.vstack(path2+random_points[i])), axis=1)
            if verbose:
                print(random_points[i])
                dtw_value, cost, DTW_map, path = dtw.dtw(pat, random_sample, return_flag = dtw.RETURN_ALL, slope_constraint=slope_constraint, window=window)
                dtw.draw_graph1d(cost, DTW_map, path, pat, random_sample)
                dtw.draw_graph1d(cost, DTW_map, combined, pat, random_sample)
            mean = np.mean([pat[combined[0]], random_sample[combined[1]]], axis=0)
            for dim in range(x.shape[2]):
                ret[i,:,dim] = np.interp(orig_steps, np.linspace(0, x.shape[1]-1., num=mean.shape[0]), mean[:,dim]).T
        else:
            if verbose > -1:
                print("There is only one pattern of class %d, skipping pattern average"%l[i])
            ret[i,:] = pat
    return jitter(ret, sigma=sigma)

In [122]:
X_train_spawner = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_train_spawner = spawner(X_train_spawner, y_train)
X_train_spawner = X_train_spawner.reshape((X_train_spawner.shape[0], X_train_spawner.shape[1]))
X_train_spawner = X_train_spawner.astype(np.float32)

100%|██████████| 57/57 [00:00<00:00, 155.36it/s]


In [123]:
# creat model and log save place,
Result_log_folder = './Example_Results_of_OS_CNN/OS_CNN_result_iter_0/'

dataset_name = "Wine_spawner"

model = OS_CNN_easy_use(
    Result_log_folder = Result_log_folder, # the Result_log_folder,
    dataset_name = dataset_name,           # dataset_name for creat log under Result_log_folder,
    device = "cuda:0",                # Gpu 
    max_epoch = 2000,                        # In our expirement the number is 2000 for keep it same with FCN for the example dataset 500 will be enough,
    paramenter_number_of_layer_list = [8*128, 5*128*256 + 2*256*128],
    )

model.fit(X_train_spawner, y_train, X_test, y_test)

y_predict = model.predict(X_test)

print('correct:',y_test)
print('predict:',y_predict)
acc = accuracy_score(y_predict, y_test)
print(acc)

code is running on  cuda:0
epoch = 49 lr =  0.001
train_acc=	 0.5263157894736842 	 test_acc=	 0.5 	 loss=	 0.8282297253608704
log saved at:
./Example_Results_of_OS_CNN/OS_CNN_result_iter_0/Wine_spawner/Wine_spawner_.txt
epoch = 99 lr =  0.001
train_acc=	 0.5087719298245614 	 test_acc=	 0.5 	 loss=	 0.19639170169830322
log saved at:
./Example_Results_of_OS_CNN/OS_CNN_result_iter_0/Wine_spawner/Wine_spawner_.txt
epoch = 149 lr =  0.001
train_acc=	 0.47368421052631576 	 test_acc=	 0.5 	 loss=	 0.023519184440374374
log saved at:
./Example_Results_of_OS_CNN/OS_CNN_result_iter_0/Wine_spawner/Wine_spawner_.txt
epoch = 199 lr =  0.001
train_acc=	 0.5263157894736842 	 test_acc=	 0.5 	 loss=	 1.0346345901489258
log saved at:
./Example_Results_of_OS_CNN/OS_CNN_result_iter_0/Wine_spawner/Wine_spawner_.txt
epoch = 249 lr =  0.001
train_acc=	 0.5263157894736842 	 test_acc=	 0.5555555555555556 	 loss=	 0.018549194559454918
log saved at:
./Example_Results_of_OS_CNN/OS_CNN_result_iter_0/Wine_spawner/Wi

In [126]:
aug_percentage = 0.2 # 20 percent
aug_length = int(aug_percentage*len(X_train_spawner))

X_train_aug = np.vstack((X_train, X_train_spawner[:aug_length]))
y_train_aug = np.concatenate((y_train, y_train[:aug_length]))
y_train_aug.shape

(68,)

In [127]:
# creat model and log save place,
Result_log_folder = './Example_Results_of_OS_CNN/OS_CNN_result_iter_0/'

dataset_name = "Wine_spawner_aug"

model = OS_CNN_easy_use(
    Result_log_folder = Result_log_folder, # the Result_log_folder,
    dataset_name = dataset_name,           # dataset_name for creat log under Result_log_folder,
    device = "cuda:0",                # Gpu 
    max_epoch = 2000,                        # In our expirement the number is 2000 for keep it same with FCN for the example dataset 500 will be enough,
    paramenter_number_of_layer_list = [8*128, 5*128*256 + 2*256*128],
    )

model.fit(X_train_aug, y_train_aug, X_test, y_test)

y_predict = model.predict(X_test)

print('correct:',y_test)
print('predict:',y_predict)
acc = accuracy_score(y_predict, y_test)
print(acc)

code is running on  cuda:0
epoch = 49 lr =  0.001
train_acc=	 0.7058823529411765 	 test_acc=	 0.5740740740740741 	 loss=	 0.657010555267334
log saved at:
./Example_Results_of_OS_CNN/OS_CNN_result_iter_0/Wine_spawner_aug/Wine_spawner_aug_.txt
epoch = 99 lr =  0.001
train_acc=	 0.6911764705882353 	 test_acc=	 0.5740740740740741 	 loss=	 0.507639467716217
log saved at:
./Example_Results_of_OS_CNN/OS_CNN_result_iter_0/Wine_spawner_aug/Wine_spawner_aug_.txt
epoch = 149 lr =  0.001
train_acc=	 0.6911764705882353 	 test_acc=	 0.5555555555555556 	 loss=	 0.465667188167572
log saved at:
./Example_Results_of_OS_CNN/OS_CNN_result_iter_0/Wine_spawner_aug/Wine_spawner_aug_.txt
epoch = 199 lr =  0.0005
train_acc=	 0.7794117647058824 	 test_acc=	 0.7962962962962963 	 loss=	 1.038734793663025
log saved at:
./Example_Results_of_OS_CNN/OS_CNN_result_iter_0/Wine_spawner_aug/Wine_spawner_aug_.txt
epoch = 249 lr =  0.00025
train_acc=	 0.75 	 test_acc=	 0.7592592592592593 	 loss=	 0.19421443343162537
log sa